# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
base_url = "https://api.geoapify.com/v1/geocode/search" 

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,salinas,36.6777,-121.6555,32.13,49,0,0.45,US,1728244281
1,1,bu'aale,1.0833,42.5833,24.54,82,95,5.03,SO,1728244282
2,2,laholm,56.5121,13.0437,8.08,87,38,2.15,SE,1728244283
3,3,mount pearl,47.5166,-52.7813,10.08,87,100,6.17,CA,1728244284
4,4,nova sintra,14.8667,-24.7167,28.09,75,70,1.89,CV,1728244286


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',          # Longitude column for x-axis
    y='Lat',          # Latitude column for y-axis
    geo=True,         # Enable geographic plotting
    tiles='OSM',      # OpenStreetMap base tiles
    size='Humidity',  # Set point size based on Humidity
    color='City',     # Set color for the points
    alpha=0.6,        # Adjust transparency
    hover_cols=['City', 'Humidity'] # Columns to display on hover
).opts(
    frame_width=800,         # Width of the plot frame
    frame_height=500)        # Height of the plot frame

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
# Define ideal weather conditions
ideal_temp_min = 22  # Minimum temperature in °C
ideal_temp_max = 28  # Maximum temperature in °C
ideal_humidity_max = 75  # Maximum humidity percentage
ideal_clouds_max = 20  # Maximum cloudiness percentage
ideal_wind_max = 10  # Maximum wind speed in km/h

# Drop any rows with null values
# Filter city_data_df based on ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) & (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Humidity"] <= ideal_humidity_max) &
    (city_data_df["Cloudiness"] <= ideal_clouds_max) &
    (city_data_df["Wind Speed"] <= ideal_wind_max)]

# Display sample data
print(f"Number of cities matching ideal conditions: {len(ideal_weather_df)}")
ideal_weather_df.head()

Number of cities matching ideal conditions: 37


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,lompoc,34.6391,-120.4579,25.94,53,2,5.14,US,1728244291
14,14,nova vicosa,-17.8919,-39.3719,23.50,67,18,3.73,BR,1728244298
18,18,alofi,-19.0595,-169.9187,24.94,73,20,7.72,NU,1728244302
19,19,acarau,-2.8856,-40.1200,26.71,72,1,8.51,BR,1728244303
37,37,jamestown,42.0970,-79.2353,26.12,49,0,5.81,US,1728244207


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,lompoc,US,34.6391,-120.4579,53,
14,nova vicosa,BR,-17.8919,-39.3719,67,
18,alofi,NU,-19.0595,-169.9187,73,
19,acarau,BR,-2.8856,-40.1200,72,
37,jamestown,US,42.0970,-79.2353,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 5000  # Radius in meters to search around the city's coordinates
params = {
    "categories": "accommodation.hotel",  # Category for hotels
    "limit": 1,  # Limit to 1 result (nearest hotel)
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL for Geoapify
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
lompoc - nearest hotel: Red Roof Inn Lompoc
nova vicosa - nearest hotel: Pousada Pontal da Barra
alofi - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
uturoa - nearest hotel: Hawaiki Nui hotel
redencao - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
sao joao da barra - nearest hotel: Pousada Mediterrâneo
lihue - nearest hotel: Kauai Palms
fallon - nearest hotel: Overland Hotel
bethel - nearest hotel: Hampton Inn Danbury
broome - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
touros - nearest hotel: Pousada Atlântico
mahina - nearest hotel: Motu Martin
piney green - nearest hotel: No hotel found
dalhart - nearest hotel: La Quinta Inn & Suites Dalhart
virginia - nearest hotel: Quirk Hotel
bundi - nearest hotel: Dev Niwas
jacupiranga - nearest hotel: No hotel found
sirte - nearest hotel: فندق المدينة

,City,Country,Lat,Lng,Humidity,Hotel Name
8,lompoc,US,34.6391,-120.4579,53,Red Roof Inn Lompoc
14,nova vicosa,BR,-17.8919,-39.3719,67,Pousada Pontal da Barra
18,alofi,NU,-19.0595,-169.9187,73,No hotel found
19,acarau,BR,-2.8856,-40.1200,72,No hotel found
37,jamestown,US,42.0970,-79.2353,49,DoubleTree Jamestown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',          # Longitude column for x-axis
    y='Lat',          # Latitude column for y-axis
    geo=True,         # Enable geographic plotting
    tiles='OSM',      # OpenStreetMap base tiles
    size='Humidity',  # Set point size based on Humidity
    color='City',     # Set color for the points
    alpha=0.6,        # Adjust transparency
    hover_cols=['City', 'Humidity'] # Columns to display on hover
).opts(
    frame_width=800,         # Width of the plot frame
    frame_height=500)        # Height of the plot frame
     
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)